# replica exchange md for mm/ml system.

I'm going to try this again...help me, god!

In [1]:
from openmmtools.testsystems import HostGuestExplicit

In [2]:
hge = HostGuestExplicit()

In [3]:
system, positions, topology = hge.system, hge.positions, hge.topology

In [4]:
from qmlify.openmm_torch.force_hybridization import HybridSystemFactory
from simtk import unit

In [5]:
hge.system.getForces()

[<openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7f9189483600> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7f91894837e0> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7f9189483960> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7f91894839f0> >,
 <openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x7f9189483a80> >]

In [6]:
from openmmtools.testsystems import HostGuestExplicit
T = 300*unit.kelvin
system, positions, topology = hge.system, hge.positions, hge.topology
system.removeForce(system.getNumForces() - 1) # remove the CMMotionRemover force because it is unknown
_atoms = list(range(126,156)) #these atoms correspond to the guest. query these with the second residue in the topology

In [7]:
system.getForces()

[<openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7f91891ebc60> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7f9189483090> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7f91894834e0> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7f9189483870> >]

In [8]:
hsf = HybridSystemFactory(topology = topology,
         alchemical_residue_indices = [1],
         system = system,
         softcore_alpha_sterics = 0.5,
         softcore_alpha_electrostatics = 0.5)
# grab the modified system and endstate system...
mod_system = hsf.system
endstate_system = hsf.endstate_system

INFO:force-hybridization:initializing system modifier...
INFO:force-hybridization:found 1 matches from [1]
INFO:force-hybridization:identified 30 to treat with ML.
INFO:force-hybridization:modifying harmonic bonds...
INFO:force-hybridization:modifying harmonic angles...
INFO:force-hybridization:modifying periodic torsions...
INFO:force-hybridization:modifying nonbondeds...
DEBUG:force-hybridization:iterating over existing exceptions and adding to custom force...
DEBUG:force-hybridization:iterating over alchemical particle combinations
DEBUG:force-hybridization:enabling rest-like scaling to alchemical particle nonbondeds


In [9]:
from openmmtools.integrators import DummyIntegrator
from simtk import openmm
T = 300. * unit.kelvin

In [10]:
nonalch_int = DummyIntegrator()
alch_int = DummyIntegrator()

In [11]:
nonalch_context, alch_context = openmm.Context(system, nonalch_int), openmm.Context(mod_system, alch_int)

In [12]:
for context in [nonalch_context, alch_context]:
    context.setPositions(positions)
    context.setPeriodicBoxVectors(*system.getDefaultPeriodicBoxVectors())

In [13]:
nonalch_context.getState(getEnergy=True).getPotentialEnergy()

Quantity(value=-51940.5232443395, unit=kilojoule/mole)

In [14]:
alch_context.getState(getEnergy=True).getPotentialEnergy()

Quantity(value=-51940.523090678435, unit=kilojoule/mole)

In [15]:
del alch_context
del nonalch_context
del nonalch_int
del alch_int

ok, now let's execute the torch alchemification wrapper.

In [16]:
from qmlify.openmm_torch.torchforce_generator import torch_alchemification_wrapper

In [17]:
ml_system, hsf_mod = torch_alchemification_wrapper(topology, system, [1], pbc=True, optimize=None)

INFO:force-hybridization:initializing system modifier...
INFO:force-hybridization:found 1 matches from [1]
INFO:force-hybridization:identified 30 to treat with ML.
INFO:force-hybridization:modifying harmonic bonds...
INFO:force-hybridization:modifying harmonic angles...
INFO:force-hybridization:modifying periodic torsions...
INFO:force-hybridization:modifying nonbondeds...
DEBUG:force-hybridization:iterating over existing exceptions and adding to custom force...
DEBUG:force-hybridization:iterating over alchemical particle combinations
DEBUG:force-hybridization:enabling rest-like scaling to alchemical particle nonbondeds
/home/dominic/anaconda3/envs/openmm_torch/lib/python3.9/site-packages/torchani/__init__.py:55: UserWarning: Dependency not satisfied, torchani.ase will not be available
  warnings.warn("Dependency not satisfied, torchani.ase will not be available")
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating c

/home/dominic/anaconda3/envs/openmm_torch/lib/python3.9/site-packages/torchani/resources/


DEBUG:torchforce_generator:elements: ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'C', 'O', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'] 
 atomic_numbers: [6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [18]:
from openmmtools.integrators import LangevinIntegrator

In [19]:
_int = LangevinIntegrator()
nonalch_context = openmm.Context(ml_system, _int)

In [20]:
nonalch_context.setPositions(hge.positions)
nonalch_context.setPeriodicBoxVectors(*ml_system.getDefaultPeriodicBoxVectors())

In [21]:
swig_params = nonalch_context.getParameters()

In [22]:
for i in swig_params:
    print(i, swig_params[i])

auxiliary_torch_scale 1.0
lambda_MM_angles 1.0
lambda_MM_bonds 1.0
lambda_MM_torsions 1.0
lambda_electrostatic_scale 0.0
lambda_epsilon_scale 0.0
lambda_nonbonded_MM_electrostatics 0.0
lambda_nonbonded_MM_sterics 0.0
lambda_scale 1.0
torch_scale 0.0


In [23]:
from openmmtools.constants import kB

In [24]:
from qmlify.openmm_torch.utils import configure_platform

In [25]:
platform = configure_platform(platform_name='CPU')

INFO:utils:conducting subsequent work with the following platform: CPU


In [26]:
beta = 1. / (kB * T)

In [27]:
from qmlify.openmm_torch.utils import compute_potential_components

In [28]:
nonalch_context.getState(getEnergy=True).getPotentialEnergy()

[W BinaryOps.cpp:595] Warning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (function operator())


Quantity(value=-51940.523090678435, unit=kilojoule/mole)

In [29]:
compute_potential_components(nonalch_context, beta, platform)

INFO:utils:conducting subsequent work with the following platform: CPU


[('HarmonicBondForce', 138.2536325732858),
 ('HarmonicAngleForce', 250.5430402178494),
 ('PeriodicTorsionForce', 127.80389899995976),
 ('NonbondedForce', -21397.72292678985),
 ('CustomBondForce', 1.2408843919367567),
 ('CustomAngleForce', 4.898738658713597),
 ('CustomTorsionForce', 29.64072359346018),
 ('CustomBondForce', 21.97856079801868),
 ('Force', 0.0)]

In [30]:
_int.step(10)

In [31]:
for i in range(100):
    _int.step(1)